In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps
from SimPEG import Utils
from SimPEG.Utils import mkvc
import SimPEG.PF as PF
import scipy as sp
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data in grid format
# workDir =r"C:\Dianne\Data\Public\GBC_Search\Phase2\Mag\AnomalousMagneticField_updatedApr17"
workDir =r"C:\Users\DominiqueFournier\Dropbox\MDRU_GIF_shared_files\GBC_Search_grd"
fName = '\MAG_UTM09.grd'

dataGrid = DataIO.loadGRDFile(workDir+fName, plotIt=False)

# 2.2.1.	Magnetic Data Response and Visualization#


## Magnetic response over the Search Phase II project area

Gridded residual magnetic data from Geoscience BC's Search Phase II project (Anomalous Magnetic Field) were downloaded from Geoscience BC's website. The magnetic data is interpolated onto a 50 $m^2$ grid. The data are shown using coordinate system NAD 83 UTM Zone 9. 


<!-- <img src="./images/SearchQuestII.png"> -->


## Define a research area

Let's choose a subset of data to view more closely. The map on the left shows the full Search Phase II magnetic dataset with a movable 'window' outlined in black, and the map on the right shows just the data encompassed by the window. The window can be resized and moved anywhere over the data using the slider bars. The default window is 40 km x 40 km in size, a size typical of surveys flown to explore magnetic responses at the exploration property, or district scale. 

In [2]:
selection = ProblemSetter.setDataExtentWidget(dataGrid)
display(selection)

interactive(children=(FloatSlider(value=669500.0, continuous_update=False, description='East', max=816075.0, m…

## Look at different filters
We can then view the data with different Fourier filters.   

In [3]:
param = Simulator.gridFiltersWidget(selection.result, gridFilter='derivativeX', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [4]:
param = Simulator.gridFiltersWidget(selection.result, gridFilter='derivativeY', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [5]:
param = Simulator.gridFiltersWidget(selection.result, gridFilter='firstVertical', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [6]:
param = Simulator.gridFiltersWidget(selection.result, gridFilter='totalHorizontal', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [7]:
param = Simulator.gridFiltersWidget(selection.result, gridFilter='tiltAngle', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [8]:
param = Simulator.gridFiltersWidget(selection.result, gridFilter='analyticSignal', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…